In [14]:
import pandas as pd 
import wandb
api = wandb.Api()

# Project is specified by <entity/project-name>
runs = api.runs("viktor-cikojevic/three_d_seg")

summary_list, config_list, name_list = [], [], []
for run in runs: 
    # .summary contains the output keys/values for metrics like accuracy.
    #  We call ._json_dict to omit large files 
    summary_list.append(run.summary._json_dict)

    # .config contains the hyperparameters.
    #  We remove special values that start with _.
    config_list.append(
        {k: v for k,v in run.config.items()
          if not k.startswith('_')})

    # .name is the human-readable name of the run.
    name_list.append(run.name)

df = pd.DataFrame({
    "summary": summary_list,
    "config": config_list,
    "name": name_list
    })


In [15]:
df.head()   

,summary,config,name
0,"{'_step': 2629, '_wandb': {'runtime': 5599}, '...","{'loss': [{'type': 'BCELoss', 'weight': 0.001}...",SMP(Unet_resnext50_32x4d_imagenet)-c512x1-bs32...
1,"{'_wandb': {'runtime': 4803}, '_runtime': 4804...","{'loss': [{'type': 'BCELoss', 'weight': 0.001}...",SMP(Unet_resnext50_32x4d_imagenet)-c512x1-bs32...
2,"{'_step': 2253, 'epoch': 0, 'train_loss': 0.00...","{'loss': [{'type': 'BCELoss', 'weight': 0.001}...",SMP(Unet_resnext50_32x4d_imagenet)-c512x1-bs32...
3,"{'train_loss': 0.014153745956718922, 'surface_...","{'loss': [{'type': 'BCELoss', 'weight': 0.001}...",SMP(Unet_resnext50_32x4d_imagenet)-c512x1-bs32...
4,"{'_step': 2253, '_wandb': {'runtime': 4810}, '...","{'loss': [{'type': 'BCELoss', 'weight': 0.001}...",SMP(Unet_resnext50_32x4d_imagenet)-c512x1-bs32...


In [16]:
group_names = []
for i in range(len(df)):
    try:
        group_name = df.config.iloc[i]['group_name']
        group_names.append(group_name)
    except:
        group_names.append('None')
    
df['group_name'] = group_names

In [17]:
df = df[df['group_name'] == 'reproduce_resnext50_32x4d'].reset_index(drop=True)
df

,summary,config,name,group_name
0,"{'_step': 2629, '_wandb': {'runtime': 5599}, '...","{'loss': [{'type': 'BCELoss', 'weight': 0.001}...",SMP(Unet_resnext50_32x4d_imagenet)-c512x1-bs32...,reproduce_resnext50_32x4d
1,"{'_wandb': {'runtime': 4803}, '_runtime': 4804...","{'loss': [{'type': 'BCELoss', 'weight': 0.001}...",SMP(Unet_resnext50_32x4d_imagenet)-c512x1-bs32...,reproduce_resnext50_32x4d
2,"{'_step': 2253, 'epoch': 0, 'train_loss': 0.00...","{'loss': [{'type': 'BCELoss', 'weight': 0.001}...",SMP(Unet_resnext50_32x4d_imagenet)-c512x1-bs32...,reproduce_resnext50_32x4d
3,"{'train_loss': 0.014153745956718922, 'surface_...","{'loss': [{'type': 'BCELoss', 'weight': 0.001}...",SMP(Unet_resnext50_32x4d_imagenet)-c512x1-bs32...,reproduce_resnext50_32x4d
4,"{'_step': 2253, '_wandb': {'runtime': 4810}, '...","{'loss': [{'type': 'BCELoss', 'weight': 0.001}...",SMP(Unet_resnext50_32x4d_imagenet)-c512x1-bs32...,reproduce_resnext50_32x4d
5,"{'lr': 8.290384721476585e-05, '_runtime': 4811...","{'loss': [{'type': 'BCELoss', 'weight': 0.001}...",SMP(Unet_resnext50_32x4d_imagenet)-c512x1-bs32...,reproduce_resnext50_32x4d
6,"{'_runtime': 4862.909891605377, 'crude_f1': 0,...","{'loss': [{'type': 'BCELoss', 'weight': 0.001}...",SMP(Unet_resnext50_32x4d_imagenet)-c512x1-bs32...,reproduce_resnext50_32x4d
7,"{'lr': 2.1113753518875455e-06, '_step': 3048, ...","{'loss': [{'type': 'BCELoss', 'weight': 0.001}...",SMP(Unet_resnext50_32x4d_imagenet)-c512x1-bs32...,reproduce_resnext50_32x4d
8,"{'train_loss': 0.011097674258053304, 'surface_...","{'loss': [{'type': 'BCELoss', 'weight': 0.001}...",SMP(Unet_resnext50_32x4d_imagenet)-c512x1-bs32...,reproduce_resnext50_32x4d


In [18]:
def flatten_dict(d, parent_key='', sep='_'):
    items = []
    if not isinstance(d, (dict, list)):
        # Return immediately if d is neither a dictionary nor a list
        return {parent_key: d}

    for k, v in d.items() if isinstance(d, dict) else enumerate(d):
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, dict):
            items.extend(flatten_dict(v, new_key, sep=sep).items())
        elif isinstance(v, list):
            for i, item in enumerate(v):
                items.extend(flatten_dict(item, f"{new_key}{sep}{i}", sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)


In [19]:
flat_configs = []
# loop over rows
for indx_row, row in df.iterrows():
    # Flatten the config   
    config = row['config']
    flat_config = flatten_dict(config)
    flat_configs.append(flat_config)

In [20]:
df = pd.DataFrame(flat_configs)
df.head()

,loss_0_type,loss_0_weight,loss_1_type,loss_1_weight,loss_2_type,loss_2_kwargs_smooth,loss_2_weight,loss_3_type,loss_3_kwargs_gamma,loss_3_weight,...,early_stopping_min_delta,batch_transform_kwargs_mixup_prob,batch_transform_kwargs_alpha_mixup,batch_transform_kwargs_cutmix_prob,batch_transform_kwargs_alpha_cutmix,experiment_name,quit_immediately,best_surface_dice,val_check_interval,apparent_batch_size
0,BCELoss,0.001,MccLoss,0.001,DiceLoss,0.0001,0.4,FocalLoss,2,0.6,...,0,0,0.4,0.3,0.4,SMP(Unet_resnext50_32x4d_imagenet)-c512x1-bs32...,False,0.880768,40000,16
1,BCELoss,0.001,MccLoss,0.001,DiceLoss,0.0001,0.4,FocalLoss,2,0.6,...,0,0,0.4,0.3,0.4,SMP(Unet_resnext50_32x4d_imagenet)-c512x1-bs32...,False,0.894861,40000,16
2,BCELoss,0.001,MccLoss,0.001,DiceLoss,0.0001,0.4,FocalLoss,2,0.6,...,0,0,0.4,0.3,0.4,SMP(Unet_resnext50_32x4d_imagenet)-c512x1-bs32...,False,0.886875,40000,16
3,BCELoss,0.001,MccLoss,0.001,DiceLoss,0.0001,0.4,FocalLoss,2,0.6,...,0,0,0.4,0.3,0.4,SMP(Unet_resnext50_32x4d_imagenet)-c512x1-bs32...,False,0.879238,40000,16
4,BCELoss,0.001,MccLoss,0.001,DiceLoss,0.0001,0.4,FocalLoss,2,0.6,...,0,0,0.4,0.3,0.4,SMP(Unet_resnext50_32x4d_imagenet)-c512x1-bs32...,False,0.884285,40000,16


In [22]:
# Remove columns that keep with the same value
columns = df.columns
for column in columns:
    if len(df[column].unique()) == 1:
        df = df.drop(column, axis=1)
# sort by best_surface_dice
df = df.sort_values(by=['best_surface_dice'], ascending=False).reset_index(drop=True)
df

,optimiser_type,optimiser_lr_scheduler_type,optimiser_lr_scheduler_kwargs_max_lr,optimiser_lr_scheduler_kwargs_pct_start,optimiser_lr_scheduler_kwargs_total_steps,experiment_name,best_surface_dice
0,SGD,OneCycleLR,0.00060,0.1,50000.0,SMP(Unet_resnext50_32x4d_imagenet)-c512x1-bs32...,0.894861
1,SGD,OneCycleLR,0.00020,0.1,50000.0,SMP(Unet_resnext50_32x4d_imagenet)-c512x1-bs32...,0.886875
2,AdamW,OneCycleLR,0.00020,0.1,50000.0,SMP(Unet_resnext50_32x4d_imagenet)-c512x1-bs32...,0.886847
3,AdamW,NaN,NaN,NaN,NaN,SMP(Unet_resnext50_32x4d_imagenet)-c512x1-bs32...,0.886645
4,AdamW,OneCycleLR,0.00060,0.1,50000.0,SMP(Unet_resnext50_32x4d_imagenet)-c512x1-bs32...,0.884285
5,SGD,OneCycleLR,0.00200,0.1,50000.0,SMP(Unet_resnext50_32x4d_imagenet)-c512x1-bs32...,0.880768
6,AdamW,OneCycleLR,0.00006,0.1,50000.0,SMP(Unet_resnext50_32x4d_imagenet)-c512x1-bs32...,0.879606
7,AdamW,OneCycleLR,0.00200,0.1,50000.0,SMP(Unet_resnext50_32x4d_imagenet)-c512x1-bs32...,0.879238
8,AdamW,OneCycleLR,0.00002,0.1,50000.0,SMP(Unet_resnext50_32x4d_imagenet)-c512x1-bs32...,0.873075
